#### Importing libraries

In [1]:
import numpy as np
import pandas as pd

#### Importing the dataset

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Removing unassigned boroughs:

In [4]:
df2 = df[df['Borough'] != 'Not assigned']

In [5]:
df2.shape

(103, 3)

##### Grouping by Postal Code and seperating neigbourhood values by commas:

In [6]:
df4 = df2.groupby('Postal Code')['Neighbourhood'].apply(', '.join).reset_index()

In [7]:
df4['Borough'] = df2['Borough']

In [8]:
df4.head()

,Postal Code,Neighbourhood,Borough
0,M1B,"Malvern, Rouge",NaN
1,M1C,"Rouge Hill, Port Union, Highland Creek",NaN
2,M1E,"Guildwood, Morningside, West Hill",North York
3,M1G,Woburn,North York
4,M1H,Cedarbrae,Downtown Toronto


In [9]:
df2.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### Filtering out neighbourhoods

In [10]:
df5 = df2[df2['Neighbourhood'] != 'Not assigned']

In [11]:
df5.shape

(103, 3)

### Importing Geospatial Data

In [20]:
lldf = pd.read_csv('https://cocl.us/Geospatial_data')

In [22]:
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Joining data frames

##### We perform an inner join to obtain a single dataframe with all details.

In [23]:
merged_inner = pd.merge(left=lldf, right=df5, left_on='Postal Code', right_on='Postal Code')

In [25]:
df_merged = merged_inner

In [26]:
df_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
